In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/archive_dataset.zip" -d "/content/dataset"

In [ ]:
import os
os.listdir('/content/dataset/Landscape Classification/Landscape Classification/Training Data')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize to fit ResNet50
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize for ResNet
])

In [ ]:
dataset_path = "/content/dataset/Landscape Classification/Landscape Classification/Training Data"

dataset_val_path = "/content/dataset/Landscape Classification/Landscape Classification/Validation Data"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

dataset_val = datasets.ImageFolder(root=dataset_val_path, transform=transform)

# Check class names
print(dataset.classes)
print(dataset_val.classes)

In [ ]:
train_size = len(dataset)  
val_size = len(dataset_val) 
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=32, shuffle=False)

In [ ]:
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for our number of classes
num_features = model.fc.in_features
num_classes = len(dataset.classes)  # Number of labels in your dataset
model.fc = torch.nn.Linear(num_features, num_classes)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")